In [ ]:
#notes
#https://developer.spotify.com/documentation/web-api/concepts/rate-limits


In [48]:
import pandas as pd
import json
import os


# Data Analysis

In [49]:
json_files = os.listdir('StreamingHistory/')
combined_json = []

for file in json_files:
    with open(f"StreamingHistory/{file}", 'r') as f:
        data = json.load(f)
        combined_json.extend(data)

df = pd.DataFrame(combined_json)
df.to_csv('all_stream_history.csv', index=False)


In [51]:
df.shape


(89608, 21)

In [4]:
df = df.rename(columns={
    'ts': 'timestamp',
    'master_metadata_track_name': 'track_name',
    'master_metadata_album_artist_name': 'artist_name',
    'master_metadata_album_album_name': 'album_name',
    'spotify_track_uri': 'track_uri'
})


In [5]:
columns_to_drop = ["username", "platform", "conn_country", "ip_addr_decrypted", "user_agent_decrypted", "episode_name", "episode_show_name",
"spotify_episode_uri", "reason_start", "reason_end", "shuffle", "skipped", "offline", "offline_timestamp", "incognito_mode"]

#"ms_played",


In [6]:
df.drop(columns=columns_to_drop, inplace=True)


In [7]:
df.to_csv('all_stream_history_2.csv', index=False)


In [8]:
data = pd.read_csv('all_stream_history_2.csv')


In [49]:
data.sample(5)


,timestamp,ms_played,track_name,artist_name,album_name,track_uri
3928,2013-09-13T04:22:01Z,2414,Just Dance - Richard Vission Remix,Lady Gaga,The Remix,spotify:track:1VodCMgoBAvamvQrD0D18o
2311,2013-05-11T08:10:51Z,2693,Objection (Tango),Shakira,Laundry Service,spotify:track:36YNa8joLwu9yor2TkZbIY
63760,2023-04-19T20:32:02Z,13580,The Most Wanted Person In The United States,100 gecs,"10,000 gecs",spotify:track:3txrp0QdO0wiF4yqiofzJv
59548,2023-01-29T13:51:31Z,29767,SPIT IN MY FACE!,ThxSoMch,SPIT IN MY FACE!,spotify:track:1N8TTK1Uoy7UvQNUazfUt5
41897,2017-07-10T18:28:17Z,199593,Gasoline,Halsey,BADLANDS,spotify:track:2IO7yf562c1zLzpanal1DT


In [28]:
tracks_to_remove = ['Deeper Sleep', 'The Early Morning Rain', 'Chicago', 'White Noise For Baby Sleep', 'White Noise For Baby Sleep']
data = data[~data['track_name'].isin(tracks_to_remove)]


In [29]:
data['track_name'].value_counts().head(10)


track_name
Lights - Single Version                                 432
Shake It Off                                            322
Kill Bill                                               136
Roar                                                    131
Let Me Go (with Alesso, Florida Georgia Line & watt)    123
Fast Car                                                118
Still into You                                          112
It Ain’t Me (with Selena Gomez)                         110
Snooze                                                  109
The Cure                                                108
Name: count, dtype: int64

In [56]:
song_search = data[data['track_name'] == 'Never Be Like You (feat. Kai)']


In [60]:
print(len(song_search))


24


In [30]:
unique_songs = data.drop_duplicates(subset=['track_name', 'artist_name'])
unique_songs = unique_songs.dropna()


In [31]:
remix_tracks = unique_songs[unique_songs['track_name'].str.contains('remix', case=False, na=False)]
unique_songs = unique_songs[~data['track_name'].str.contains('remix', case=False, na=False)]


/var/folders/t6/bfyvsvq54r1f1_xd_gjqncpr0000gn/T/ipykernel_4733/89475970.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unique_songs = unique_songs[~data['track_name'].str.contains('remix', case=False, na=False)]


In [39]:
unique_songs = unique_songs[unique_songs['ms_played'] >= 180000]
unique_songs.to_csv('filtered_dataset.csv', index=False)


In [45]:
unique_songs.sample(10)


NameError: name 'unique_songs' is not defined

# Spotify API

In [160]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


In [53]:
fdata = pd.read_csv('all_stream_history_2.csv')


In [55]:
fdata.shape


(89608, 6)

In [71]:
fdata.sample(10)


,timestamp,ms_played,track_name,artist_name,album_name,track_uri
63946,2023-04-26T05:18:11Z,188866,She Wolf,Shakira,She Wolf (Expanded Edition),spotify:track:4cS2HQ6jK80vqdY9ofpztx
1567,2012-11-12T10:49:08Z,196719,Boys (feat. Pharrell Williams) - Co-Ed Remix,Britney Spears,Greatest Hits: My Prerogative,spotify:track:4FNjramiHyil63p2rtH5Vt
9281,2013-12-06T16:03:35Z,3436,Plaintain Bay,Timaya,Nigeria Club,spotify:track:0dHMhpJYDrKOeHMyCeL9yn
18533,2018-06-18T14:18:13Z,189959,Maps,Maroon 5,V,spotify:track:4gbVRS8gloEluzf0GzDOFc
12365,2014-01-03T02:51:28Z,230225,Almost,Tamia,Between Friends,spotify:track:7oeAmjYo4MJDklImlVSavw
48670,2017-11-16T23:21:56Z,1557,Tu me manques,Harry Diboula,Entre vous et moi (32 Hits),spotify:track:2tjdUXAQO53oMqpRoA4bwP
44805,2017-08-20T00:30:25Z,53041,Kissin' On My Tattoos,August Alsina,Testimony,spotify:track:4MOXruB3ARc4PePI6IkGO7
18617,2018-06-19T01:11:19Z,0,Feelings,Maroon 5,V,spotify:track:22TcZGXKvZ8XnhnnaRcaiB
652,2012-07-01T20:26:56Z,233072,Dog Days Are Over,Florence + The Machine,Lungs,spotify:track:1YLJVmuzeM2YSUkCCaTNUB
46638,2017-10-08T01:17:21Z,198237,How Long,Charlie Puth,How Long,spotify:track:32DGGj6KlNuBr6WaqRxpxi


In [169]:
from dotenv import load_dotenv
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')


In [170]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))


In [171]:
track_uri = 'spotify:track:32DGGj6KlNuBr6WaqRxpxi'
track_id = track_uri.split(':')[-1]
track_info = sp.track(track_id)

print("Track Name:", track_info['name'])
print("Artists:", ', '.join([artist['name'] for artist in track_info['artists']]))
print("Album:", track_info['album']['name'])
print("Release Date:", track_info['album']['release_date'])
print("Popularity:", track_info['popularity'])


Track Name: How Long
Artists: Charlie Puth
Album: How Long
Release Date: 2017-10-05
Popularity: 0


In [172]:
sp.audio_features('4sOX1nhpKwFWPvoMMExi3q')


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=4sOX1nhpKwFWPvoMMExi3q:
 Max Retries, reason: too many 429 error responses

In [164]:
import time

def get_audio_features(track_uris, max_retries=5):
    features = []
    for uri in track_uris:
        retries = 0
        while retries < max_retries:
            try:
                audio_features = sp.audio_features(uri)
                if audio_features and audio_features[0]:
                    feature = audio_features[0]
                    features.append({
                        'track_uri': uri,
                        'danceability': feature['danceability'],
                        'energy': feature['energy'],
                        'tempo': feature['tempo'],
                        'valence': feature['valence'],
                        'loudness': feature['loudness'],
                        'speechiness': feature['speechiness'],
                        'acousticness': feature['acousticness'],
                        'instrumentalness': feature['instrumentalness'],
                        'liveness': feature['liveness'],
                        'key': feature['key'],
                        'mode': feature['mode']
                    })
                break
            except spotipy.exceptions.SpotifyException as e:
                if e.http_status == 429:
                    retries += 1
                    wait_time = int(e.headers.get("Retry-After", 1)) * (2 ** (retries - 1))
                    print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    raise e
    return features


In [166]:

track_uris = ["4sOX1nhpKwFWPvoMMExi3q"]
audio_features = get_audio_features(track_uris)
print(audio_features)


Max Retries reached


Rate limit exceeded. Retrying in 1 seconds...


Max Retries reached


Rate limit exceeded. Retrying in 2 seconds...


KeyboardInterrupt: 

In [152]:
unique_track_ids = fdata['track_uri'].unique()
len(unique_track_ids)


25113

In [153]:
track_ids = [i.split(':')[-1] for i in unique_track_ids if isinstance(i, str)]


In [154]:
type(track_ids)


list

In [163]:
audio_features = get_audio_features(track_ids)
features_df = pd.DataFrame(audio_features)
fdata = fdata.merge(features_df, on='track_uri', how='left')


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2jgSexPzkIxLecTorGgjh6:
 Max Retries, reason: too many 429 error responses